<a href="https://colab.research.google.com/github/LeeGitaek/2020_AI_Class/blob/master/RNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim 
from torch.autograd import Variable
import numpy as np 

In [3]:
n_hidden = 50
learning_rate = 0.01
epochs = 2000 

string = "hello pytorch.how long can a rnn cell remember?"# show us your limit!"
chars = "abcdefghijklmnopqrstuvwxyz ?!.,:;01"
char_list = [i for i in chars]
char_len = len(char_list)
n_letters = len(char_list)

In [4]:
def string_to_onehot(string):
    start = np.zeros(shape=len(char_list),dtype=int)
    end   = np.zeros(shape=len(char_list),dtype=int)
    start[-2] = 1
    end[-1] = 1
    for i in string:
        idx = char_list.index(i)
        zero = np.zeros(shape=char_len,dtype=int)
        zero[idx] = 1
        start = np.vstack([start,zero])
    output = np.vstack([start,end])
    return output

In [5]:
def onehot_to_word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)
    return char_list[onehot.argmax()]

In [6]:
# RNN with 1 hidden layer


class RNN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(RNN,self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)

        self.act_fn = nn.Tanh()

    def forward(self,input,hidden):
        hidden = self.act_fn(self.i2h(input)+self.h2h(hidden))
        output = self.i2o(hidden)
        return output,hidden

    def init_hidden(self):
        return Variable(torch.zeros(1,self.hidden_size))


rnn = RNN(n_letters,n_hidden,n_letters)


In [8]:
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

In [11]:
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for epoch in range(epochs):
    rnn.zero_grad()
    total_loss = 0
    hidden = rnn.init_hidden()

    for j in range(one_hot.size()[0]-1):
        input = Variable(one_hot[j:j+1,:])
        output, hidden = rnn.forward(input, hidden)
        target = Variable(one_hot[j+1])
        loss = loss_func(output.view(-1),target.view(-1))
        total_loss += loss
        input = output

    total_loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print('epoch {:.4f}  / cost = {:.6f}'.format(epoch,total_loss))

epoch 0.0000  / cost = 0.000211
epoch 10.0000  / cost = 0.000187
epoch 20.0000  / cost = 0.000075
epoch 30.0000  / cost = 0.000067
epoch 40.0000  / cost = 0.000160
epoch 50.0000  / cost = 0.000249
epoch 60.0000  / cost = 0.000372
epoch 70.0000  / cost = 0.000117
epoch 80.0000  / cost = 0.000173
epoch 90.0000  / cost = 0.000116
epoch 100.0000  / cost = 0.000062
epoch 110.0000  / cost = 0.000096
epoch 120.0000  / cost = 0.000153
epoch 130.0000  / cost = 0.000096
epoch 140.0000  / cost = 0.000119
epoch 150.0000  / cost = 0.000020
epoch 160.0000  / cost = 0.000060
epoch 170.0000  / cost = 0.000248
epoch 180.0000  / cost = 0.000117
epoch 190.0000  / cost = 0.000614
epoch 200.0000  / cost = 0.000480
epoch 210.0000  / cost = 0.000342
epoch 220.0000  / cost = 0.000158
epoch 230.0000  / cost = 0.000060
epoch 240.0000  / cost = 0.000027
epoch 250.0000  / cost = 0.000018
epoch 260.0000  / cost = 0.000067
epoch 270.0000  / cost = 0.000071
epoch 280.0000  / cost = 0.000123
epoch 290.0000  / cost = 

In [12]:
hidden = rnn.init_hidden()
input = Variable(one_hot[0:1,:])

for i in range(len(string)):
    output, hidden = rnn.forward(input, hidden)
    print(onehot_to_word(output.data),end="")
    input = output

hello pytorch.how long can a rnn cell remember?